In [68]:
import pandas as pd
import numpy as np
from datetime import datetime, time
import math

In [69]:
route_df = pd.read_csv("route.csv")
route_df['count'] = None
route_df['mean'] = None
route_df['std'] = None
route_df['min_threshold'] = None
route_df['max_threshold'] = None
print(len(route_df))
route_df.head(5)

25


,rd_id,route,source,source_lat,source_long,destination,destination_lat,destination_long,distance,distance_direct,count,mean,std,min_threshold,max_threshold
0,0,6,Wagle Bus Depot,19.201610,72.945940,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,None,None,None,None,None
1,1,6,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,Jost Vasahat,19.202385,72.949391,120,174.27,None,None,None,None,None
2,2,6,Jost Vasahat,19.202385,72.949391,Indira Nagar,19.204769,72.949893,270,270.31,None,None,None,None,None
3,3,6,Indira Nagar,19.204769,72.949893,Savarkar Nagar,19.204484,72.953639,400,394.55,None,None,None,None,None
4,4,6,Savarkar Nagar,19.204484,72.953639,Kamgar Hospital Thane,19.203513,72.955577,230,230.37,None,None,None,None,None


In [70]:
trip_df = pd.read_csv("trip_df_final.csv")
print(len(trip_df))
trip_df.head(5)

5525


,rd_id,route,source,source_lat,source_long,destination,destination_lat,destination_long,distance,distance_direct,trip_id,ref_no,source_time,destination_time,duration,weather,day_type,shift_type,source_accuracy,destination_accuracy
0,0,6,Wagle Bus Depot,19.20161,72.94594,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,1,-1,-1,-1,-1,NaN,NaN,NaN,-1.000000,-1.000000
1,0,6,Wagle Bus Depot,19.20161,72.94594,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,220,17,03-08-24 6:06,03-08-24 6:07,46,NaN,NaN,NaN,5.450204,24.303749
2,0,6,Wagle Bus Depot,19.20161,72.94594,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,3,-1,-1,-1,-1,NaN,NaN,NaN,-1.000000,-1.000000
3,0,6,Wagle Bus Depot,19.20161,72.94594,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,217,52,04-08-24 8:24,04-08-24 8:25,76,NaN,NaN,NaN,12.591176,23.451745
4,0,6,Wagle Bus Depot,19.20161,72.94594,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,214,52,04-08-24 13:39,04-08-24 13:39,45,NaN,NaN,NaN,33.209382,15.362254


In [71]:
SPEED_THRESHOULD = 1 # In m/s
trip_df.drop(trip_df[trip_df['duration'] == -1].index, inplace=True)
trip_df['time'] = pd.to_datetime(trip_df['source_time']).dt.time
trip_df['speed'] = (trip_df['distance'] + trip_df['source_accuracy'] - trip_df['destination_accuracy']) / trip_df['duration']
trip_df['actual_duration'] = np.ceil(trip_df['distance'] / trip_df['speed'])
trip_df.sort_values(["time"], inplace=True)
trip_df.drop(trip_df[trip_df["speed"] <= SPEED_THRESHOULD].index, inplace=True)
print(len(trip_df))
trip_df.head(5)

1459


C:\Users\Shubh Patel\AppData\Local\Temp\ipykernel_18528\3892210727.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trip_df['time'] = pd.to_datetime(trip_df['source_time']).dt.time


,rd_id,route,source,source_lat,source_long,destination,destination_lat,destination_long,distance,distance_direct,...,destination_time,duration,weather,day_type,shift_type,source_accuracy,destination_accuracy,time,speed,actual_duration
153,0,6,Wagle Bus Depot,19.201610,72.945940,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,400,328.41,...,24-08-24 5:43,105,NaN,NaN,NaN,8.850893,28.786904,05:41:00,3.619657,111.0
302,1,6,Annabhau Sathe Chowk (Thane)/Sathe Nagar,19.200869,72.948967,Jost Vasahat,19.202385,72.949391,120,174.27,...,24-08-24 5:44,36,NaN,NaN,NaN,28.786904,22.130059,05:43:00,3.518246,35.0
523,2,6,Jost Vasahat,19.202385,72.949391,Indira Nagar,19.204769,72.949893,270,270.31,...,24-08-24 5:45,60,NaN,NaN,NaN,22.130059,8.223065,05:44:00,4.731783,58.0
744,3,6,Indira Nagar,19.204769,72.949893,Savarkar Nagar,19.204484,72.953639,400,394.55,...,24-08-24 5:46,91,NaN,NaN,NaN,8.223065,26.209654,05:45:00,4.197950,96.0
965,4,6,Savarkar Nagar,19.204484,72.953639,Kamgar Hospital Thane,19.203513,72.955577,230,230.37,...,24-08-24 5:47,35,NaN,NaN,NaN,26.209654,48.375924,05:46:00,5.938107,39.0


In [72]:
for i in range(len(route_df)):
    station_pair_id = pd.to_numeric(route_df.iloc[i]["rd_id"])
    station_pair_df = trip_df[trip_df["rd_id"] == station_pair_id]
    station_pair_summary = station_pair_df["actual_duration"].describe()
    # if station_pair_count > 1:
    station_pair_count = station_pair_summary["count"]
    station_pair_mean = round(0 if pd.isna(station_pair_summary["mean"]) else station_pair_summary["mean"])
    station_pair_std = round(0 if pd.isna(station_pair_summary["std"]) else station_pair_summary["std"])
    station_pair_min_threshould = math.floor(station_pair_mean - station_pair_std)
    station_pair_max_threshould = math.ceil(station_pair_mean + station_pair_std)
    route_df.at[route_df.index[i], "count"] = station_pair_count
    route_df.at[route_df.index[i], "mean"] = station_pair_mean
    route_df.at[route_df.index[i], "std"] = station_pair_std
    route_df.at[route_df.index[i], "min_threshold"] = station_pair_min_threshould
    route_df.at[route_df.index[i], "max_threshold"] = station_pair_max_threshould

In [73]:
route_df.head(5)
route_df.to_csv("route_df_processed.csv")